In [55]:
import pickle
import os

In [ ]:
with os.open("method.py", "w+") as f:
    f.write()

In [49]:
import tarfile

In [50]:
import tarfile
with tarfile.open("sample.tar", "w:gz") as tar:
    for name in ["foo", "bar", "quux"]:
        tar.add(name)

FileNotFoundError: [Errno 2] No such file or directory: 'foo'

In [ ]:
!pip3 install argo-workflows

In [3]:
# !pip3 install kubernetes

In [4]:
# !pip3 install openapi3

 # Dockerize Python Application

In [ ]:
FROM python:3
ADD my_script.py /
CMD [ "python", "./my_script.py" ]

# Task With Input Artifact

In [93]:
import uuid

from argo.workflows.client.models import *

container = V1Container(name=str(uuid.uuid4()), image="debian:9.4", command=["sh", "-c"], args=["kubectl version"])

input_artifact = V1alpha1Artifact(name="kubectl", path="/usr/local/bin/kubectl", mode=755)

inputs = V1alpha1Inputs(artifacts=[input_artifact])

template = V1alpha1Template(name="kubectl-input-artifact", container=container, inputs=inputs)

metadata = V1ObjectMeta(generate_name="arguments-artifacts-")

artifact = V1alpha1Artifact(name="kubectl", http=V1alpha1HTTPArtifact("https://storage.googleapis.com/kubernetes-release/release/v1.8.0/bin/linux/amd64/kubectl"))

arguments = V1alpha1Arguments(artifacts=[artifact])

spec = V1alpha1WorkflowSpec(entrypoint="kubectl-input-artifact", templates=[template], arguments=arguments)

status = V1alpha1WorkflowStatus()
        
workflow = V1alpha1Workflow(
    api_version="argoproj.io/v1alpha1",
    kind="Workflow",
    metadata=metadata, 
    spec=spec,
    status=status)

v1alpha1.create_namespaced_workflow('default', workflow)

{'api_version': 'argoproj.io/v1alpha1',
 'kind': 'Workflow',
 'metadata': {'annotations': None,
              'cluster_name': None,
              'creation_timestamp': datetime.datetime(2019, 12, 16, 15, 5, 38, tzinfo=tzutc()),
              'deletion_grace_period_seconds': None,
              'deletion_timestamp': None,
              'finalizers': None,
              'generate_name': 'arguments-artifacts-',
              'generation': 1,
              'initializers': None,
              'labels': None,
              'managed_fields': None,
              'name': 'arguments-artifacts-dqpsk',
              'namespace': 'default',
              'owner_references': None,
              'resource_version': '53140',
              'self_link': '/apis/argoproj.io/v1alpha1/namespaces/default/workflows/arguments-artifacts-dqpsk',
              'uid': '7d61f5f0-695f-42b4-8301-85e13704c6c0'},
 'spec': {'active_deadline_seconds': None,
          'affinity': None,
          'arguments': {'artifacts':